In [ ]:
import os
from PIL import Image
import torch
import pandas as pd
from transformers import AutoImageProcessor, AutoModelForObjectDetection
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# YOLO 모델 및 프로세서 불러오기
image_processor = AutoImageProcessor.from_pretrained("valentinafeve/yolos-fashionpedia")
model = AutoModelForObjectDetection.from_pretrained("valentinafeve/yolos-fashionpedia")

# 이미지가 저장된 기본 경로 (대폴더 경로)
base_save_path = r'C:\Users\usert\Desktop\무신사_프로젝트\스냅사진'

# 처리된 이미지를 저장할 기본 경로 (대폴더 경로)
output_base_path = r'C:\Users\usert\Desktop\무신사_프로젝트\스냅사진_전처리'

# 바운딩 박스를 조정하는 함수
def adjust_box(box, scale_factor=1.1, image_size=(0, 0)):
    xmin, ymin, xmax, ymax = box
    width = xmax - xmin
    height = ymax - ymin

    new_xmin = max(0, xmin - (scale_factor - 1) * width / 2)
    new_ymin = max(0, ymin - (scale_factor - 1) * height / 2)
    new_xmax = min(image_size[0], xmax + (scale_factor - 1) * width / 2)
    new_ymax = min(image_size[1], ymax + (scale_factor - 1) * height / 2)

    return [new_xmin, new_ymin, new_xmax, new_ymax]

# DataFrame을 저장할 리스트 초기화
data = []

# 하위 폴더 내 모든 이미지에 대해 객체 탐지, 자르기, 저장
def process_images_in_subfolders(base_path, output_path):
    for subfolder in os.listdir(base_path):
        subfolder_path = os.path.join(base_path, subfolder)

        if os.path.isdir(subfolder_path):
            # 해당 하위 폴더의 출력 경로 설정
            relative_path = os.path.relpath(subfolder_path, base_path)
            output_folder = os.path.join(output_path, relative_path)
            os.makedirs(output_folder, exist_ok=True)

            for root, dirs, files in os.walk(subfolder_path):
                for file in files:
                    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                        input_image_path = os.path.join(root, file)

                        # 이미지 로드
                        image = Image.open(input_image_path)

                        # 이미지 모드를 확인하고, RGB로 변환
                        if image.mode != "RGB":
                            image = image.convert("RGB")

                        # YOLO 모델로 객체 탐지
                        inputs = image_processor(images=image, return_tensors="pt")
                        outputs = model(**inputs)
                        target_sizes = torch.tensor([image.size[::-1]])
                        results = image_processor.post_process_object_detection(outputs, threshold=0.4, target_sizes=target_sizes)[0]

                        # 상의에 해당하는 라벨들
                        priority_labels = {0, 1, 2, 3, 4}  # 우선 확인할 상의 라벨들
                        secondary_labels = {5, 9, 10, 11}  # 두 번째로 확인할 상의 라벨들

                        best_box = None

                        # 1. 우선순위 라벨들에서 가장 먼저 발견된 박스 출력
                        for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
                            if label.item() in priority_labels:  # 0, 1, 2, 3, 4에 해당하는 라벨만 필터링
                                box = [round(i, 2) for i in box.tolist()]
                                best_box = adjust_box(box, scale_factor=1.2, image_size=image.size)  # 바운딩 박스 확장 적용
                                break  # 첫 번째로 발견된 박스를 선택하고 반복 종료

                        # 2. 우선순위 라벨이 없으면, 두 번째 라벨들 중 가장 큰 박스를 선택
                        if not best_box:
                            max_area = 0
                            for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
                                if label.item() in secondary_labels:  # 5, 9, 10, 11에 해당하는 라벨만 필터링
                                    box = [round(i, 2) for i in box.tolist()]
                                    adjusted_box = adjust_box(box, scale_factor=1.2, image_size=image.size)  # 바운딩 박스 확장 적용

                                    # 현재 바운딩 박스의 면적 계산
                                    xmin, ymin, xmax, ymax = adjusted_box
                                    current_area = (xmax - xmin) * (ymax - ymin)
                                    
                                    # 가장 큰 바운딩 박스를 선택
                                    if current_area > max_area:
                                        max_area = current_area
                                        best_box = adjusted_box

                        # 상의가 검출된 경우에만 데이터 저장
                        if best_box:
                            # 데이터 리스트에 추가 (폴더명, 파일명, 좌표)
                            data.append({
                                "Folder": relative_path,
                                "File": file,
                                "Coordinates": best_box
                            })

                            # 서브플롯 설정
                            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 10))
                            ax1.imshow(image)

                            # 첫 번째 서브플롯: 원본 이미지와 바운딩 박스
                            for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
                                box = [round(i, 2) for i in box.tolist()]
                                adjusted_box = adjust_box(box, scale_factor=1.3, image_size=image.size)
                                xmin, ymin, xmax, ymax = adjusted_box
                                width, height = xmax - xmin, ymax - ymin
                                rect = patches.Rectangle((xmin, ymin), width, height, linewidth=2, edgecolor='red', facecolor='none')
                                ax1.add_patch(rect)
                                ax1.text(xmin, ymin - 10, f'{model.config.id2label[label.item()]}: {round(score.item(), 3)}', color='red', fontsize=12, weight='bold')

                            ax1.axis('off')
                            ax1.set_title('Detecting')

                            # 두 번째 서브플롯: 선택된 바운딩 박스를 자르고 표시
                            cropped_image = image.crop(best_box)
                            ax2.imshow(cropped_image)
                            ax2.axis('off')
                            ax2.set_title('Cutting box')

                            # 각 서브플롯을 파일로 저장
                            output_cropped_path = os.path.join(output_folder, f"{os.path.splitext(file)[0]}_cropped.png")
                            cropped_image.save(output_cropped_path)

                            plt.close(fig)
                            print(f"Processed and saved images: {output_cropped_path}")
                        else:
                            print(f"No top detected in image: {input_image_path}. Image not saved.")

# 함수 호출하여 모든 하위 폴더 이미지 처리
process_images_in_subfolders(base_save_path, output_base_path)

# DataFrame 생성 및 CSV 파일로 저장
df = pd.DataFrame(data)


print("All images have been processed and saved.")
